In [14]:
from batchgenerators.utilities.file_and_folder_operations import *
import shutil

In [16]:
#vol_base='/home/ldh/1_Tindex/data/T_Index_nii_test/volume/'
vol_base='/home/ldh/1_Tindex/data/T_Index_nii_test_B/volume/'
for v in subfiles(vol_base):
    file_name=v.split('/')[-1].split('0000')[0][:-1]
    print(file_name)
    
    mask_path=v.replace('volume','mask').replace('_0000','')
    pred_path=v.replace('volume','prediction').replace('_0000','')
    
    mask_base=vol_base.replace('volume','T_Index_extract_mask')+'/'+file_name
    maybe_mkdir_p(mask_base)
    
    pred_base=vol_base.replace('volume','T_Index_extract_pred')+'/'+file_name
    maybe_mkdir_p(pred_base)
    
    shutil.copy(v, mask_base+'/imaging.nii.gz')
    shutil.copy(mask_path, mask_base+'/segmentation.nii.gz')
    
    shutil.copy(v, pred_base+'/imaging.nii.gz')
    shutil.copy(pred_path, pred_base+'/segmentation.nii.gz')

458_8794304_181227_preop
459_3034805_181224_preop
462_8035121_190108_preop
464_8279838_190122_preop
481_8856525_190404_preop
485_8855172_190409_preop
542_10097654_190928_preop
573_101585253_200108_preop
574_10166362_200130_preop
576_10148339_191211_preop
580_10140888_200204_preop
583_10160375_200120_preop
584_5718731_200320_preop
590_5496283_200429_preop
591_10207803_200514_preop
592-10179272-200119-preop
593_6043739_200601_preop
594_10171022_200114_preop


In [17]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import ConvexHull
from scipy.ndimage import binary_erosion
from skimage.measure import label
from skimage.morphology import convex_hull_image
from batchgenerators.utilities.file_and_folder_operations import *
import os

In [18]:
def sinus_seg(seg1, seg2, seg_ori_data):
    seg_dataset = []
    for seg_kid in [seg1,seg2]:
        x,y,z = np.where(seg_kid==1)
        
        x_min = max(min(x)-5,0)
        y_min = max(min(y)-5,0)
        z_min = max(min(z)-5,0)
        
        x_max = min(max(x)+5, seg_kid.shape[0])
        y_max = min(max(y)+5, seg_kid.shape[1])
        z_max = min(max(z)+5, seg_kid.shape[2])                                   
        
        seg_kid_crop = seg_kid[x_min:x_max, y_min:y_max, z_min:z_max]
        crop_result = convex_hull_image(seg_kid_crop.astype('uint8'))
        
        result = np.zeros(seg_kid.shape)
        result[x_min:x_max, y_min:y_max, z_min:z_max] = crop_result
        
        seg_result = binary_erosion(result,iterations=3)
        values = [0,4]
        seg_result *= np.isin(seg_ori_data, values)
        seg_result = getLargestCC(seg_result)
        seg_dataset.append(seg_result)
        
    seg_new_data = np.zeros(seg_kid.shape)
    seg_new_data[seg_dataset[0]==1] = 1
    seg_new_data[seg_dataset[1]==1] = 1
    
    return seg_new_data

In [19]:
def getLargestCC(segmentation):
    labels = label(segmentation)
    if labels.max() ==0:
        return 0
    else:
        #assert( labels.max() != 0 ) # assume at least 1 CC
        largestCC = labels == np.argmax(np.bincount(labels.flat)[1:])+1
        return largestCC

In [20]:
def sinus_seg_main(seg_path, save_path):
    if not isfile(save_path):
        # 1. data load 
        seg = nib.load(seg_path)
        seg_data = seg.get_fdata()
        seg_ori_data = seg_data.copy()
        seg_data[seg_data!=1]=0

        # 2. seg1, seg2 : left, right kidney
        labels = label(seg_data)
        #assert len(np.unique(labels))==3
        region_list=[]
        for b in range(1,len(np.unique(labels))):
            isl = labels==b
            region = len(np.where(isl==True)[0])
            region_list.append(region)
        region_list_sort = np.argsort(region_list)

        seg1 = labels==region_list_sort[-1]+1
        seg2 = labels==region_list_sort[-2]+1

        # 3. sinus seg
        result = sinus_seg(seg1, seg2, seg_ori_data)
        seg_ori_data[result==1]=4
        seg_ori_data[seg_ori_data==1]=0
        seg_ori_data[seg1==1]=1
        seg_ori_data[seg2==1]=1

        vol = nib.Nifti1Image(seg_ori_data,seg.affine)
        nib.save(vol, save_path)
    else:
        pass
    
    print(save_path)

In [10]:
data_path = '/home/ldh/1_Tindex/data/T_Index_nii'
folders = subfolders(data_path, join=False)
seg_path=[]
save_path=[]
for fold in folders:
    seg_path.append(data_path+'/'+fold+'/segmentation.nii.gz')
    save_path.append(data_path+'/'+fold+'/segmentation_sinus.nii.gz')

In [11]:
from multiprocessing import Pool
p = Pool(8)
p.starmap_async(sinus_seg_main, zip(seg_path,save_path))
p.close()
p.join()

/home/ldh/1_Tindex/data/T_Index_nii/001/segmentation_sinus.nii.gz/home/ldh/1_Tindex/data/T_Index_nii/033/segmentation_sinus.nii.gz/home/ldh/1_Tindex/data/T_Index_nii/023/segmentation_sinus.nii.gz/home/ldh/1_Tindex/data/T_Index_nii/010/segmentation_sinus.nii.gz/home/ldh/1_Tindex/data/T_Index_nii/061/segmentation_sinus.nii.gz/home/ldh/1_Tindex/data/T_Index_nii/041/segmentation_sinus.nii.gz/home/ldh/1_Tindex/data/T_Index_nii/055/segmentation_sinus.nii.gz/home/ldh/1_Tindex/data/T_Index_nii/069/segmentation_sinus.nii.gz






/home/ldh/1_Tindex/data/T_Index_nii/003/segmentation_sinus.nii.gz
/home/ldh/1_Tindex/data/T_Index_nii/035/segmentation_sinus.nii.gz/home/ldh/1_Tindex/data/T_Index_nii/026/segmentation_sinus.nii.gz/home/ldh/1_Tindex/data/T_Index_nii/044/segmentation_sinus.nii.gz/home/ldh/1_Tindex/data/T_Index_nii/063/segmentation_sinus.nii.gz/home/ldh/1_Tindex/data/T_Index_nii/016/segmentation_sinus.nii.gz
/home/ldh/1_Tindex/data/T_Index_nii/070/segmentation_sinus.nii.gz
/home/ldh/1_Tin

# Test

In [21]:
def sinus_seg_main(seg_path, save_path):
    try:
        # 1. data load 
        seg = nib.load(seg_path)
        seg_data = seg.get_fdata()
        seg_ori_data = seg_data.copy()
        seg_data[seg_data!=1]=0

        # 2. seg1, seg2 : left, right kidney
        labels = label(seg_data)
        #assert len(np.unique(labels))==3
        region_list=[]
        for b in range(1,len(np.unique(labels))):
            isl = labels==b
            region = len(np.where(isl==True)[0])
            region_list.append(region)
        region_list_sort = np.argsort(region_list)

        seg1 = labels==region_list_sort[-1]+1
        seg2 = labels==region_list_sort[-2]+1

        # 3. sinus seg
        result = sinus_seg(seg1, seg2, seg_ori_data)
        seg_ori_data[result==1]=4
        seg_ori_data[seg_ori_data==1]=0
        seg_ori_data[seg1==1]=1
        seg_ori_data[seg2==1]=1

        vol = nib.Nifti1Image(seg_ori_data,seg.affine)
        nib.save(vol, save_path)
    except:
        print(save_path)

In [22]:
#data_path = '/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred'
data_path = '/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_pred'
folders = subfolders(data_path, join=False)
seg_path=[]
save_path=[]
for fold in folders:
    seg_path.append(data_path+'/'+fold+'/segmentation.nii.gz')
    save_path.append(data_path+'/'+fold+'/segmentation_sinus.nii.gz')

In [23]:
for a,b in zip(seg_path, save_path):
    sinus_seg_main(a,b)

In [24]:
#data_path = '/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred'
data_path = '/home/ldh/1_Tindex/data/T_Index_nii_test_A/T_Index_extract_mask'
folders = subfolders(data_path, join=False)
seg_path=[]
save_path=[]
for fold in folders:
    seg_path.append(data_path+'/'+fold+'/segmentation.nii.gz')
    save_path.append(data_path+'/'+fold+'/segmentation_sinus.nii.gz')
for a,b in zip(seg_path, save_path):
    sinus_seg_main(a,b)

In [25]:
#data_path = '/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred'
data_path = '/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_pred'
folders = subfolders(data_path, join=False)
seg_path=[]
save_path=[]
for fold in folders:
    seg_path.append(data_path+'/'+fold+'/segmentation.nii.gz')
    save_path.append(data_path+'/'+fold+'/segmentation_sinus.nii.gz')
for a,b in zip(seg_path, save_path):
    sinus_seg_main(a,b)

In [26]:
#data_path = '/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred'
data_path = '/home/ldh/1_Tindex/data/T_Index_nii_test_B/T_Index_extract_mask'
folders = subfolders(data_path, join=False)
seg_path=[]
save_path=[]
for fold in folders:
    seg_path.append(data_path+'/'+fold+'/segmentation.nii.gz')
    save_path.append(data_path+'/'+fold+'/segmentation_sinus.nii.gz')
for a,b in zip(seg_path, save_path):
    sinus_seg_main(a,b)

In [12]:
#data_path = '/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred'
data_path = '/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_pred_4'
folders = subfolders(data_path, join=False)
seg_path=[]
save_path=[]
for fold in folders:
    seg_path.append(data_path+'/'+fold+'/segmentation.nii.gz')
    save_path.append(data_path+'/'+fold+'/segmentation_sinus.nii.gz')
for a,b in zip(seg_path, save_path):
    sinus_seg_main(a,b)

In [13]:
#data_path = '/home/ldh/1_Tindex/data/T_Index_nii_test/T_Index_extract_pred'
data_path = '/home/ldh/1_Tindex/data/T_Index_nii_final_test/T_Index_extract_mask'
folders = subfolders(data_path, join=False)
seg_path=[]
save_path=[]
for fold in folders:
    seg_path.append(data_path+'/'+fold+'/segmentation.nii.gz')
    save_path.append(data_path+'/'+fold+'/segmentation_sinus.nii.gz')
for a,b in zip(seg_path, save_path):
    sinus_seg_main(a,b)